## DDQN in Hockey Enviroment - Evaluation

Evaluate a trained DDQN against various environments

Base DQN implementation adapted from HW7

In [1]:
from importlib import reload

import numpy as np
from tqdm.notebook import tqdm

import DDQN.DDQN as ddqn
from DDQN.DDQN import DDQNAgent
from DDQN.evaluation import compare_agents, display_stats
import hockey.hockey_env as h_env

reload(h_env)
reload(ddqn)

/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


<module 'DDQN.DDQN' from '/home/kivanc/ders/RL/project/RL-Hockey/DDQN/DDQN.py'>

In [2]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

## Environment & Agent Initialization

In [3]:
env = h_env.HockeyEnv(mode=h_env.Mode.NORMAL)
env.reset()

(array([-3.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  3.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.37346315,  0.74111128,  0.        ,
         0.        ,  0.        ,  0.        ]),
 {'winner': 0,
  'reward_closeness_to_puck': 0.0,
  'reward_touch_puck': 0.0,
  'reward_puck_direction': 0.0})

In [4]:
STATE_PATH = "./DDQN/models/hockey_mixed_dev/"

agent_player = DDQNAgent(
    env.observation_space,
    env.discrete_action_space
)
agent_player.load_state(STATE_PATH)

agent_opp_weak = h_env.BasicOpponent(weak=True)
agent_opp_strong = h_env.BasicOpponent(weak=False)

## Evaluation

### Winning Rates Against Opponents & Match Statistics

In [6]:
weak_opp_stats = compare_agents(
    agent_player, agent_opp_weak, env, num_matches=1000, tqdm=tqdm
)
strong_opp_stats = compare_agents(
    agent_player, agent_opp_strong, env, num_matches=1000, tqdm=tqdm
)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
print("Weak Opponent Stats:")
display_stats(weak_opp_stats)

print("Strong Opponent Stats:")
display_stats(strong_opp_stats)

Weak Opponent Stats:
Player Observation Mean:
  x pos player one: -2.030508588284232
  y pos player one: 0.5147690643466569
  angle player one: 0.08989587131911483
  x vel player one: 0.4264605116501769
  y vel player one: 0.3203428443802452
  angular vel player one: 0.04945519872067978
  x player two: 2.8218203605041383
  y player two: -0.019924067602875864
  angle player two: -0.004754417855837733
  y vel player two: 0.0325562418307977
  y vel player two: -0.012695931352237835
  angular vel player two: 0.0014306109641619318
  x pos puck: 0.04673915050150489
  y pos puck: 0.0055823536817285095
  x vel puck: -0.9200172461770368
  y vel puck: -0.04205428590664245
  left player puck keep time: 0.8536769651026661
  right player puck keep time: 0.8125275035495146

Relative Std. Change in Agent Observations:
  x pos player one: 0.16425445998432814
  y pos player one: 0.5107164377696342
  angle player one: 0.7743364145456684
  x vel player one: -0.24060218230283872
  y vel player one: 0.5461

### Rendered Demonstration Against Strong Opponent

In [ ]:
obs_buffer = []
reward_buffer = []
obs, _ = env.reset()
obs_opp = env.obs_agent_two()

done = False
trunc = False
step = 0
while not (done or trunc):
    step += 1
    env.render()

    a1_discr = agent_player.act(obs)
    a1 = env.discrete_to_continous_action(a1_discr)
    a2 = agent_opp_strong.act(obs_opp)

    obs, r, done, trunc, _ = env.step(np.hstack([a1, a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)

    obs_opp = env.obs_agent_two()

    if done or trunc:
        print(f"Episode done in {step} steps")
        break

Episode done in 116 steps


In [ ]:
env.close()